In [1]:
import math
import numpy as np

# 使用するデータ

天気 | 風速 | 湿度 || 花火 \
ーーーーーーーーーー\
晴れ | 強い | 高い || No\
曇り | 弱い | 高い || Yes\
曇り | 強い | 低い || No\
晴れ | 弱い | 高い || Yes\
雨　 | 弱い | 高い || No

In [2]:
labels_name = ['天気','風速','湿度']
labels = [['晴れ', '曇り', '曇り', '晴れ' , '雨' ],
         ['強い', '弱い', '強い', '弱い' , '弱い'],
        ['高い', '高い', '低い', '高い' , '高い']]
target   = ['No' , 'Yes' , 'No' , 'Yes' , 'No' ]

# ID3アルゴリズム

In [3]:
class ID3():   
    def get_entropy(self,target):
        """
        エントロピーを求める。
        入力
            target：1次元のndarray。各データのクラスを並べたもの
        出力
           entropy：float。入力データに対するエントロピーを返す。

        例)
           target  = ['No','Yes','No','Yes','No']
           =>2/5*log2(5/2) + 3/5*log2(5/3) = 0.972
        """
        classes = set(target)
        class_num = len(classes)
        element_num = len(target)
        entropy = 0
        for i in classes:
            i_num = np.count_nonzero(target==i)
            p_i = i_num/element_num
            entropy -= p_i * math.log2(p_i)
        return entropy
    
    def get_information_gain(self,label, target):
        """
        labelで分類した場合の、エントロピーを求める
        入力
            label ：1次元のndarray。各データの属性値を並べたもの
            target：1次元のndarray。各データのクラスを並べたもの
        出力
           entropy：float。入力データに対する情報ゲインを返す。

        例)
           label  = ['晴れ', '曇り','曇り','晴れ' ,'雨'] 
           target = ['No' , 'Yes' ,'No' ,'Yes' ,'No']
           =>0.972 - 0.8 = 0.172

        """
        origin_entropy = self.get_entropy(target)
        labels = set(label)
        element_num = len(target)
        entropy = 0.
        for i in labels:
            i_index = [j for j, x in enumerate(label) if x == i]#該当ラベルのデータのindexのリスト
            p_i = len(i_index)/element_num
            label_i = label[i_index]
            target_i = target[i_index]
            entropy += p_i * self.get_entropy(target_i)
        return origin_entropy - entropy
    
    def select_column(self,labels, target):
        """
        情報ゲインが最大のラベルを求める
        入力
            label ：ndarray。各データの属性値を並べたもの
            target：1次元のndarray。各データのクラスを並べたもの
        出力
           entropy：int。情報ゲイン(識別力)の最も高いラベルのindexを返す

        例)
           label[0] = ['晴れ', '曇り', '曇り', '晴れ' , '雨' ] : 天気
           label[1] = ['強い', '弱い', '強い', '弱い' , '弱い'] :風速
           label[2] = ['高い', '高い', '低い', '高い' , '高い'] :湿度
           target   = ['No' , 'Yes' , 'No' , 'Yes' , 'No' ]
           =>情報gainは、 0.172, 0.423, 0.172となり、

        注)　各ラベルで分類前のエントロピーは一定のため、
           実際には、各ラベルで分類後のエントロピーが最大のものを選んだ方が、計算コストは低い
        """
        gains = list()
        for i in labels:
            gains.append(self.get_information_gain(i,target))
        gains = np.array(gains)
        max_index = np.argmax(gains)
        return max_index    
    
    def decision_tree(self, labels, target,labels_name,depth=1):
        """
        ID3アルゴリズムによる決定木を出力する。
        再起的に実行する
        入力
            labels ：ndarray。各データの属性値を並べたもの
            target：1次元のndarray。各データのクラスを並べたもの
            labels_nane：labelsにおける、各属性の名前
            depth：int。木の深さ。※表示のインデントをつけるため
        出力
           -

        """
        next_index = self.select_column(labels,target)#分類に使用する属性を決める
        print("　"*depth,'#',labels_name[next_index])
        elems = set(labels[next_index])
        for i in elems:
            i_index = [j for j, x in enumerate(labels[next_index]) if x == i]#該当ラベルのデータのindexのリスト
            label_i = labels[:,i_index]
            target_i = target[i_index]
            if np.all(target_i == target_i[0]):
                class_name = target_i[0]
                print("　"*(depth+1),'-',i,':',class_name)
                continue
            print("　"*(depth+1),'-',i,)
            self.decision_tree(label_i,target_i,labels_name, depth+2)

In [4]:
def print_decision_tree_by_ID3(labels_name,labels, target):
    """
    ID3アルゴリズムによる決定木を表示する
     入力
        labels ：list。各データの属性値を並べたもの
        target：list。各データのクラスを並べたもの
        labels_nane：list。labelsにおける、各属性の名前
    """
    labels_name = np.array(labels_name)
    labels = np.array(labels)
    target = np.array(target)
    
    model = ID3()
    
    model.decision_tree(labels,target,labels_name)

In [5]:
print_decision_tree_by_ID3(labels_name,labels,target)

　 # 風速
　　 - 弱い
　　　 # 天気
　　　　 - 曇り : Yes
　　　　 - 雨 : No
　　　　 - 晴れ : Yes
　　 - 強い : No


---
## 出力について
出力結果は、
- \#がクラス
- \-が属性値を表す

インデントで、木の深さを表している

---
---


# CARTアルゴリズム

In [6]:
class CART():
    def get_gini_impurity(self,target):
        """
        ジニ不純度を求める。
        入力
            target：1次元のndarray。各データのクラスを並べたもの
        出力
           gini：float。入力データに対するジニ不純度を返す。

        例)
           target  = ['No','Yes','No','Yes','No']
           => 2/5(1 - 2/5) + 3/5(1 - 3/5) = 0.48
        """
        classes = set(target)
        class_num = len(classes)
        element_num = len(target)
        gini = 0
        for i in classes:
            i_num = np.count_nonzero(target==i)
            p_i = i_num/element_num
            gini += p_i * (1 - p_i)
        return gini
    
    def get_information_gain(self, label, atribute, target):
        """
        labelで分類した場合の、エントロピーを求める
        入力
            label ：1次元のndarray。各データの属性値を並べたもの
            atribute：string。情報ゲインを計算する属性値。
            target：1次元のndarray。各データのクラスを並べたもの
        出力
           entropy：float。入力データに対する情報ゲインを返す。

        例)
           label  = ['晴れ', '曇り','曇り','晴れ' ,'雨']
           atribute = '晴れ'
           target = ['No' , 'Yes' ,'No' ,'Yes' ,'No']
           => 0.48 - ( 2/5*0.5 + 3/5*0.44444) = 0.13333

        """
        origin_gini = self.get_gini_impurity(target)
        element_num = len(target)
        gini = 0.
        #属性値=atributeについて
        i_index = [j for j, x in enumerate(label) if x == atribute]#該当ラベルのデータのindexのリスト
        p_i = len(i_index)/element_num
        label_i = label[i_index]
        target_i = target[i_index]
        gini += p_i * self.get_gini_impurity(target_i)
        #その他について
        i_index = [j for j, x in enumerate(label) if x != atribute]#該当ラベルのデータのindexのリスト
        p_i = len(i_index)/element_num
        label_i = label[i_index]
        target_i = target[i_index]
        gini += p_i * self.get_gini_impurity(target_i)
        return origin_gini - gini
    
    def select_column(self, labels, target):
        """
        情報ゲインが最大のラベルを求める
        入力
            labels ：ndarray。各データの属性値を並べたもの
            target：1次元のndarray。各データのクラスを並べたもの
        出力
           index：int。情報ゲイン(識別力)の最も高いラベルのindex
            atribute:str。情報ゲイン(識別力)の最も高い属性値を返す

        例)
           labels[0] = ['晴れ', '曇り', '曇り', '晴れ' , '雨' ] : 天気
           labels[1] = ['強い', '弱い', '強い', '弱い' , '弱い'] :風速
           labels[2] = ['高い', '高い', '低い', '高い' , '高い'] :湿度
           target   = ['No' , 'Yes' , 'No' , 'Yes' , 'No' ]
           =>情報gainは、 0.172, 0.423, 0.172となり、

        注)　属性値が2種類の属性に対して、下記のコードは実際には冗長である。(どちらか1つについて情報ゲインを計算すれば本来良い)
        """
        gains = list()
        for i,l in enumerate(labels):
            ls = list(set(l))#その属性における属性値のユニークなリスト
            for j in ls:
                gains.append([i,j,self.get_information_gain(l,j,target)])#[属性(labelsのindex), 属性値, 情報ゲイン]
        gains = np.array(gains)
        max_index = np.argmax(gains[:,2])
        return int(gains[max_index,0]),gains[max_index,1]
    
    def decision_tree(self, labels, target,labels_name,depth=1):
        """
        ID3アルゴリズムによる決定木を出力する。
        再起的に実行する
        入力
            labels ：ndarray。各データの属性値を並べたもの
            target：1次元のndarray。各データのクラスを並べたもの
            labels_nane：labelsにおける、各属性の名前
            depth：int。木の深さ。※表示のインデントをつけるため
        出力
           -

        """
        next_index,atribute = self.select_column(labels,target)#分類に使用する属性を決める
        print("　"*depth,'#',labels_name[next_index])
        #今回分類に使用する属性値のデータについて
        i_index = [j for j, x in enumerate(labels[next_index]) if x == atribute]#該当ラベルのデータのindexのリスト
        label_i = labels[:,i_index]
        target_i = target[i_index]
        if np.all(target_i == target_i[0]):
            class_name = target_i[0]
            print("　"*(depth+1),'-',atribute,':',class_name)
        else:
            print("　"*(depth+1),'-',atribute,)
            self.decision_tree(label_i,target_i,labels_name, depth+2)

        #それ以外のデータについて
        i_index = [j for j, x in enumerate(labels[next_index]) if x != atribute]#該当ラベルのデータのindexのリスト
        label_i = labels[:,i_index]
        target_i = target[i_index]
        if np.all(target_i == target_i[0]):
            class_name = target_i[0]
            print("　"*(depth+1),'-','NOT'+atribute,':',class_name)
        else:
            print("　"*(depth+1),'-','NOT'+atribute,)
            self.decision_tree(label_i,target_i,labels_name, depth+2)

In [7]:
def print_decision_tree_by_CART(labels_name,labels, target):
    """
    ID3アルゴリズムによる決定木を表示する
     入力
        labels ：list。各データの属性値を並べたもの
        target：list。各データのクラスを並べたもの
        labels_nane：list。labelsにおける、各属性の名前
    """
    labels_name = np.array(labels_name)
    labels = np.array(labels)
    target = np.array(target)
    
    model = CART()
    
    model.decision_tree(labels,target,labels_name)

In [8]:
print_decision_tree_by_CART(labels_name,labels,target)

　 # 風速
　　 - 弱い
　　　 # 天気
　　　　 - 雨 : No
　　　　 - NOT雨 : Yes
　　 - NOT弱い : No
